In [ ]:
# Hive to Unity Catalog permissions mapping
hive_to_uc_mapping = {
    "SELECT": "SELECT",
    "CREATE": "CREATE TABLE",  # Adjust this as needed for different CREATE types
    "OWN": "ALL PRIVILEGES",
    "READ_METADATA": "USE SCHEMA",
    "MODIFY": "MODIFY",
    "USAGE": "USE SCHEMA",
    "CREATE_NAMED_FUNCTION": "CREATE FUNCTION"
}

# Specify the catalog name
catalog_name = "stg_one_dt_datalake"

# Read the hive_metastore_grants table into a DataFrame
grants_df = sqlContext.sql("SELECT * FROM hive_metastore_grants")

# Collect the data into a list for iteration
grants_data = grants_df.collect()

# Initialize lists to store success and error logs
success_log = []
error_log = []

# Iterate through each row and construct the GRANT SQL command
for row in grants_data:
    try:
        schema = row.Schema
        principal = row.Principal
        hive_action_type = row.ActionType

        # Change principal from 'users' to 'account users'
        if principal == 'users':
            principal = 'account users'

        # Map Hive action type to Unity Catalog equivalent
        uc_action_type = hive_to_uc_mapping.get(hive_action_type, hive_action_type)

        # Construct the GRANT SQL command with the catalog name and mapped action type
        grant_command = f"GRANT {uc_action_type} ON SCHEMA `{catalog_name}`.`{schema}` TO `{principal}`"

        # Execute the GRANT SQL command
        sqlContext.sql(grant_command)

        # Log the successful operation
        success_log.append(f"Success: {grant_command}")

    except Exception as e:
        # Log the error and continue
        error_log.append(f"Error processing {schema}: {str(e)}")

# After the loop, print successes and errors in tabular format
print("Successful Operations:")
for success in success_log:
    print(success)

print("\nErrors Encountered:")
for error in error_log:
    print(error)
